In [2]:
import os
from dataclasses import dataclass
from enum import Enum
from PIL import Image, ImageDraw, ImageFont
from typing import List, Optional, Tuple
import sys


class TypeCarte(Enum):
    IA = "C'est de l'IA ! ✨"
    NON_IA = "Ce n'est pas de l'IA 🧠"


class Categorie(Enum):
    QUOTIDIEN = "QUOTIDIEN"
    PROFESSIONNEL = "PROFESSIONNEL"
    TECHNIQUE = "TECHNIQUE"


@dataclass
class CarteCriteres:
    titre: str
    categorie: Categorie
    type_carte: TypeCarte
    explication: str
    chemin_image: Optional[str] = None
    
    def __post_init__(self):
        # Vérifier la longueur de l'explication
        if len(self.explication) > 200:
            raise ValueError("L'explication ne doit pas dépasser 200 caractères")


class GenerateurCartes:
    """Classe pour générer des cartes pour le jeu 'Le Détective IA'"""
    
    def __init__(self, dossier_sortie: str = "cartes_generees"):
        self.dossier_sortie = dossier_sortie
        self.largeur = 400
        self.hauteur = 600
        self.marge = 20
        self.police_titre = None
        self.police_categorie = None
        self.police_type = None
        self.police_explication = None
        
        # Couleurs
        self.couleur_fond = (255, 255, 255)  # Blanc
        self.couleur_categorie = (59, 130, 246)  # Bleu comme dans l'image
        self.couleur_texte = (0, 0, 0)  # Noir
        
        # Créer le dossier de sortie s'il n'existe pas
        os.makedirs(self.dossier_sortie, exist_ok=True)
        
        # Charger les polices (à adapter selon les polices disponibles)
        self._charger_polices()
    
    def _charger_polices(self):
        """Charge les polices nécessaires pour générer les cartes"""
        # Utilisation de polices par défaut pour éviter les problèmes de disponibilité
        self.police_categorie = ImageFont.load_default()
        self.police_titre = ImageFont.load_default()
        self.police_type = ImageFont.load_default()
        self.police_explication = ImageFont.load_default()
        
        # Vous pouvez activer cette partie si vous avez des polices spécifiques
        # try:
        #     # Ces chemins doivent être adaptés selon votre système
        #     self.police_categorie = ImageFont.truetype("arial.ttf", 24)
        #     self.police_titre = ImageFont.truetype("arial_bold.ttf", 28)
        #     self.police_type = ImageFont.truetype("arial_bold.ttf", 22)
        #     self.police_explication = ImageFont.truetype("arial.ttf", 18)
        # except Exception as e:
        #     print(f"Erreur lors du chargement des polices: {e}")
        #     print("Utilisation des polices par défaut")
    
    def generer_carte(self, criteres: CarteCriteres, nom_fichier: str = None) -> Image.Image:
        """Génère une carte avec les critères spécifiés"""
        # Créer une nouvelle image
        img = Image.new('RGB', (self.largeur, self.hauteur), self.couleur_fond)
        draw = ImageDraw.Draw(img)
        
        # Dessiner l'en-tête de catégorie
        draw.rectangle(
            [(0, 0), (self.largeur, 80)],
            fill=self.couleur_categorie
        )
        
        # Ajouter le texte de catégorie
        self._centrer_texte(
            draw, 
            self.largeur // 2, 
            40, 
            criteres.categorie.value, 
            self.police_categorie,
            (255, 255, 255)  # Blanc
        )
        
        # Ajouter le titre
        self._centrer_texte(
            draw, 
            self.largeur // 2, 
            120, 
            criteres.titre, 
            self.police_titre,
            self.couleur_texte
        )
        
        # Dessiner l'espace pour l'image
        rect_img_y1 = 160
        rect_img_y2 = 380
        draw.rectangle(
            [(self.marge, rect_img_y1), (self.largeur - self.marge, rect_img_y2)],
            fill=(240, 240, 240),  # Gris clair
            outline=(200, 200, 200)  # Gris pour la bordure
        )
        
        # Ajouter l'image si disponible
        if criteres.chemin_image and os.path.exists(criteres.chemin_image):
            try:
                image_carte = Image.open(criteres.chemin_image)
                # Redimensionner l'image pour qu'elle tienne dans l'espace
                image_carte.thumbnail((self.largeur - 2 * self.marge, rect_img_y2 - rect_img_y1))
                # Centrer l'image
                x_offset = (self.largeur - image_carte.width) // 2
                y_offset = rect_img_y1 + (rect_img_y2 - rect_img_y1 - image_carte.height) // 2
                img.paste(image_carte, (x_offset, y_offset))
            except Exception as e:
                print(f"Erreur lors du chargement de l'image {criteres.chemin_image}: {e}")
                # Ajouter un texte de remplacement
                self._centrer_texte(
                    draw, 
                    self.largeur // 2, 
                    (rect_img_y1 + rect_img_y2) // 2, 
                    "[Image non disponible]", 
                    self.police_explication,
                    (150, 150, 150)  # Gris moyen
                )
        else:
            # Dessiner un placeholder pour l'image
            self._centrer_texte(
                draw, 
                self.largeur // 2, 
                (rect_img_y1 + rect_img_y2) // 2, 
                "200\n×\n150", 
                self.police_explication,
                (150, 150, 150)  # Gris moyen
            )
        
        # Dessiner une ligne de séparation
        draw.line(
            [(self.marge, 430), (self.largeur - self.marge, 430)],
            fill=(200, 200, 200),  # Gris clair
            width=1
        )
        
        # Ajouter le type (IA ou non IA)
        self._centrer_texte(
            draw, 
            self.largeur // 2, 
            470, 
            criteres.type_carte.value, 
            self.police_type,
            self.couleur_texte
        )
        
        # Ajouter l'explication
        self._texte_multiligne(
            draw,
            self.marge,
            510,
            criteres.explication,
            self.police_explication,
            self.couleur_texte,
            self.largeur - 2 * self.marge
        )
        
        # Sauvegarder l'image si un nom de fichier est spécifié
        if nom_fichier:
            chemin_complet = os.path.join(self.dossier_sortie, nom_fichier)
            img.save(chemin_complet)
            print(f"Carte sauvegardée: {chemin_complet}")
        
        return img
    
    def _centrer_texte(self, draw, x, y, texte, police, couleur):
        """Centre le texte à la position (x, y)"""
        if '\n' in texte:
            # Texte multiligne centré
            lignes = texte.split('\n')
            hauteur_ligne = police.getbbox("Ay")[3]  # Approximation de la hauteur
            y_offset = y - (hauteur_ligne * len(lignes)) // 2
            
            for ligne in lignes:
                bbox = police.getbbox(ligne)
                largeur_texte = bbox[2] - bbox[0]
                draw.text((x - largeur_texte // 2, y_offset), ligne, font=police, fill=couleur)
                y_offset += hauteur_ligne
        else:
            # Texte sur une seule ligne
            bbox = police.getbbox(texte)
            largeur_texte = bbox[2] - bbox[0]
            hauteur_texte = bbox[3] - bbox[1]
            draw.text((x - largeur_texte // 2, y - hauteur_texte // 2), texte, font=police, fill=couleur)
    
    def _texte_multiligne(self, draw, x, y, texte, police, couleur, largeur_max):
        """Dessine du texte multiligne avec retour à la ligne automatique"""
        mots = texte.split()
        lignes = []
        ligne_courante = ""
        
        for mot in mots:
            test_ligne = f"{ligne_courante} {mot}".strip()
            bbox = police.getbbox(test_ligne)
            largeur_test = bbox[2] - bbox[0]
            
            if largeur_test <= largeur_max:
                ligne_courante = test_ligne
            else:
                lignes.append(ligne_courante)
                ligne_courante = mot
        
        if ligne_courante:
            lignes.append(ligne_courante)
        
        hauteur_ligne = police.getbbox("Ay")[3]  # Approximation de la hauteur
        
        for i, ligne in enumerate(lignes):
            draw.text((x, y + i * (hauteur_ligne + 4)), ligne, font=police, fill=couleur)


def generer_exemples():
    """Génère les exemples de cartes pour le jeu"""
    generateur = GenerateurCartes()
    
    # Définir quelques exemples de cartes
    exemples = [
        # Exemples IA - Quotidien
        CarteCriteres(
            titre="Suggestions Netflix",
            categorie=Categorie.QUOTIDIEN,
            type_carte=TypeCarte.IA,
            explication="Utilise le machine learning pour analyser vos préférences et suggérer du contenu personnalisé."
        ),
        CarteCriteres(
            titre="Assistants vocaux",
            categorie=Categorie.QUOTIDIEN,
            type_carte=TypeCarte.IA,
            explication="Utilise le traitement du langage naturel pour comprendre vos questions et s'adapter à vos préférences."
        ),
        CarteCriteres(
            titre="Filtres Instagram",
            categorie=Categorie.QUOTIDIEN,
            type_carte=TypeCarte.IA,
            explication="Utilise la vision par ordinateur pour détecter les visages et appliquer des effets contextuels intelligents."
        ),
        
        # Exemples Non IA - Quotidien
        CarteCriteres(
            titre="Calculatrice",
            categorie=Categorie.QUOTIDIEN,
            type_carte=TypeCarte.NON_IA,
            explication="Suit des règles mathématiques prédéfinies sans apprentissage ni adaptation."
        ),
        CarteCriteres(
            titre="Réveil/alarme",
            categorie=Categorie.QUOTIDIEN,
            type_carte=TypeCarte.NON_IA,
            explication="Fonctionne selon des paramètres fixes que vous définissez, sans capacité d'apprentissage ou d'adaptation."
        ),
        CarteCriteres(
            titre="Galerie photo",
            categorie=Categorie.QUOTIDIEN,
            type_carte=TypeCarte.NON_IA,
            explication="Trie chronologiquement vos images selon des règles simples sans analyser leur contenu."
        ),
        
        # Exemples IA - Professionnel
        CarteCriteres(
            titre="Gmail - Réponses suggérées",
            categorie=Categorie.PROFESSIONNEL,
            type_carte=TypeCarte.IA,
            explication="Analyse le contenu des emails pour générer des réponses pertinentes en fonction du contexte."
        ),
        CarteCriteres(
            titre="LinkedIn - Suggestions",
            categorie=Categorie.PROFESSIONNEL,
            type_carte=TypeCarte.IA,
            explication="Utilise des algorithmes d'apprentissage pour recommander des connexions et du contenu pertinent."
        ),
        CarteCriteres(
            titre="Outils de recrutement IA",
            categorie=Categorie.PROFESSIONNEL,
            type_carte=TypeCarte.IA,
            explication="Analyse les CV et les profils pour présélectionner les candidats selon des critères appris."
        ),
        
        # Exemples Non IA - Professionnel
        CarteCriteres(
            titre="Excel - Formules simples",
            categorie=Categorie.PROFESSIONNEL,
            type_carte=TypeCarte.NON_IA,
            explication="Applique des formules mathématiques prédéfinies sans apprentissage ni adaptation au contexte."
        ),
        CarteCriteres(
            titre="Scanner de documents",
            categorie=Categorie.PROFESSIONNEL,
            type_carte=TypeCarte.NON_IA,
            explication="Convertit simplement les documents physiques en fichiers numériques sans analyse intelligente."
        ),
        CarteCriteres(
            titre="Bases de données simples",
            categorie=Categorie.PROFESSIONNEL,
            type_carte=TypeCarte.NON_IA,
            explication="Stocke et récupère des informations selon des requêtes prédéfinies sans apprentissage."
        ),
        
        # Exemples IA - Technique
        CarteCriteres(
            titre="Reconnaissance faciale",
            categorie=Categorie.TECHNIQUE,
            type_carte=TypeCarte.IA,
            explication="Utilise des réseaux de neurones pour identifier des visages et leurs caractéristiques uniques."
        ),
        CarteCriteres(
            titre="Traduction automatique",
            categorie=Categorie.TECHNIQUE,
            type_carte=TypeCarte.IA,
            explication="Comprend le contexte linguistique pour traduire entre différentes langues avec précision."
        ),
        CarteCriteres(
            titre="Détection de fraude",
            categorie=Categorie.TECHNIQUE,
            type_carte=TypeCarte.IA,
            explication="Analyse les comportements d'achat pour identifier les transactions suspectes et apprend des nouvelles fraudes."
        ),
        
        # Exemples Non IA - Technique
        CarteCriteres(
            titre="Compression de fichiers",
            categorie=Categorie.TECHNIQUE,
            type_carte=TypeCarte.NON_IA,
            explication="Applique des algorithmes de compression fixes sans apprentissage ni adaptation au contenu."
        ),
        CarteCriteres(
            titre="Pare-feu basique",
            categorie=Categorie.TECHNIQUE,
            type_carte=TypeCarte.NON_IA,
            explication="Bloque des ports ou des adresses IP spécifiques selon des règles prédéfinies."
        ),
        CarteCriteres(
            titre="Correcteur orthographique simple",
            categorie=Categorie.TECHNIQUE,
            type_carte=TypeCarte.NON_IA,
            explication="Compare les mots à un dictionnaire statique sans comprendre le contexte ou la grammaire."
        )
    ]
    
    # Générer les cartes
    for exemple in exemples:
        nom_fichier = f"{exemple.categorie.value.lower()}_{exemple.titre.replace(' ', '_').lower()}.png"
        generateur.generer_carte(exemple, nom_fichier)


# Interface utilisateur simplifiée
class DetectiveIAInterface:
    """Interface utilisateur simplifiée pour le générateur de cartes"""
    
    def __init__(self):
        self.generateur = GenerateurCartes()
    
    def creer_nouvelle_carte(self):
        """Interface pour créer une nouvelle carte"""
        print("=== Création d'une nouvelle carte ===")
        
        # Sélection de la catégorie
        print("\nCatégories disponibles:")
        for i, cat in enumerate(Categorie):
            print(f"{i+1}. {cat.value}")
        
        choix_cat = int(input("Sélectionnez une catégorie (numéro): ")) - 1
        categorie = list(Categorie)[choix_cat]
        
        # Sélection du type
        print("\nTypes disponibles:")
        for i, typ in enumerate(TypeCarte):
            print(f"{i+1}. {typ.value}")
        
        choix_type = int(input("Sélectionnez un type (numéro): ")) - 1
        type_carte = list(TypeCarte)[choix_type]
        
        # Autres informations
        titre = input("\nTitre de la carte: ")
        explication = input("Explication (200 caractères max): ")
        chemin_image = input("Chemin de l'image (ou laissez vide): ").strip() or None
        
        # Création de la carte
        try:
            criteres = CarteCriteres(
                titre=titre,
                categorie=categorie,
                type_carte=type_carte,
                explication=explication,
                chemin_image=chemin_image
            )
            
            nom_fichier = f"{categorie.value.lower()}_{titre.replace(' ', '_').lower()}.png"
            self.generateur.generer_carte(criteres, nom_fichier)
            print(f"\nCarte créée avec succès: {nom_fichier}")
        
        except Exception as e:
            print(f"Erreur lors de la création de la carte: {e}")
    
    def executer(self):
        """Exécute l'interface utilisateur"""
        while True:
            print("\n=== Menu Détective IA ===")
            print("1. Créer une nouvelle carte")
            print("2. Générer les exemples par défaut")
            print("3. Quitter")
            
            choix = input("Votre choix: ")
            
            if choix == "1":
                self.creer_nouvelle_carte()
            elif choix == "2":
                generer_exemples()
                print("Exemples générés avec succès!")
            elif choix == "3":
                print("Au revoir!")
                break
            else:
                print("Choix invalide, veuillez réessayer.")


In [3]:
if __name__ == "__main__":
    # Vous pouvez utiliser l'interface
    # interface = DetectiveIAInterface()
    # interface.executer()
    
    # Ou générer simplement les exemples
    generer_exemples()

Carte sauvegardée: cartes_generees\quotidien_suggestions_netflix.png
Carte sauvegardée: cartes_generees\quotidien_assistants_vocaux.png
Carte sauvegardée: cartes_generees\quotidien_filtres_instagram.png
Carte sauvegardée: cartes_generees\quotidien_calculatrice.png


c:\Users\flori\.conda\envs\py3113\Lib\ast.py:50: RuntimeWarning: coroutine 'generer_exemples' was never awaited
  return compile(source, filename, mode, flags,


FileNotFoundError: [Errno 2] No such file or directory: 'F:\\github\\Ollama\\cartes_generees\\quotidien_réveil\\alarme.png'